In [1]:
from casadi import *
import casadi
import numpy as np
import pandas


In [2]:
Iner_body = np.array([[0.0158533, -3.66e-5, -6.11e-5],
                      [-3.66e-5, 0.0377999, -2.75e-5],
                      [-6.11e-5, -2.75e-5, 0.0456542]])
hip_offset = np.array([0.1805, 0.047, 0])
mass_body_body = 6.0
mass_body_leg = 1.935
inertia_body = Iner_body + 4 * mass_body_leg * np.diag([
    hip_offset[1]**2 + hip_offset[2]**2, hip_offset[0]**2 + hip_offset[2]**2,
    hip_offset[0]**2 + hip_offset[1]**2
])
mass_body = mass_body_body + 4 * mass_body_leg
mass_body = SX([mass_body])
inertia_body = SX(inertia_body)
gravity = SX([9.81])

t = SX.sym("t")
f = SX.sym('f', 12)
foot_p = SX.sym('f_p', 12)
p = SX.sym('p', 3)
p_dot = SX.sym('p_dot', 3)
mass = SX.sym('m')
m = diag(vcat([mass, mass, mass]))
m = substitute(m, mass, mass_body)
I = inertia_body
g = vertcat(0, 0, -gravity)
theta = SX.sym("theta", 3)
omega = SX.sym("omega", 3)
Omega = skew(omega)

R1 = SX(3, 3)
ct0 = cos(theta[0])
st0 = sin(theta[0])

ct1 = cos(theta[1])
st1 = sin(theta[1])

ct2 = cos(theta[2])
st2 = sin(theta[2])

R1[0, :] = vcat([ct2, -st2, 0])
R1[1, :] = vcat([st2, ct2, 0])
R1[2, :] = vcat([0, 0, 1])

R2 = SX(3, 3)
R2[0, :] = vcat([ct1, 0, st1])
R2[1, :] = vcat([0, 1, 0])
R2[2, :] = vcat([-st1, 0, ct1])

R3 = SX(3, 3)
R3[0, :] = vcat([1, 0, 0])
R3[1, :] = vcat([0, ct0, -st0])
R3[2, :] = vcat([0, st0, ct0])

R = R1 @ R2 @ R3

J_inv = SX(3, 3)
sT2 = sin(theta[2])
cT2 = cos(theta[2])
cT1 = cos(theta[1])
sT2 = sin(theta[2])
tT1 = tan(theta[1])

J_inv[0, :] = vcat([cT2 / cT1, sT2 / cT1, 0])
J_inv[1, :] = vcat([-sT2, cT2, 0])
J_inv[2, :] = vcat([cT2 * tT1, sT2 * tT1, 1])


In [3]:
rhs = SX(12, 1)
rhs[0:3] = p_dot
rhs[3:6] = J_inv @ R @ omega

sum_forces = SX([0, 0, 0])
for i in range(4):
    sum_forces += f[i * 3:(i + 1) * 3]
rhs[6:9] = inv(m) @ sum_forces + g

sum_force_mom = SX([0, 0, 0])
for i in range(4):
    sum_force_mom += skew(foot_p[i * 3:(i + 1) * 3] - p) @ f[i * 3:(i + 1) * 3]
# rhs[9:12] = R @ inv(R @ I @R.T) @ R.T @ (sum_force_mom)
rhs[9:12] = inv(I) @ (R.T @ sum_force_mom - skew(omega) @ I @ omega)
# rhs[9:12] = inv(I) @ (sum_force_mom)
# rhs[9:12] = (sum_force_mom)

f_rhs = Function("rhs", [p, theta, p_dot, omega, foot_p, f], [rhs])


In [4]:
x = SX.sym("x", 12)
u = SX.sym("u", 24)
ode = f_rhs(x[0:3], x[3:6], x[6:9], x[9:12], u[:12], u[12:24])

s = np.linspace(1, 10, 12)
p = np.linspace(0.4, 4, 24)
print(f_rhs(s[0:3], s[3:6], s[6:9], s[9:12], p[:12], p[12:24]).full())
%timeit f_rhs(s[0:3], s[3:6], s[6:9], s[9:12], p[:12], p[12:24])


[[  5.90909091]
 [  6.72727273]
 [  7.54545455]
 [-17.87348989]
 [ -5.65715127]
 [ 28.9946412 ]
 [  0.86829947]
 [  0.91386621]
 [ -8.85056705]
 [-79.87327881]
 [ 96.99820883]
 [ -2.50032409]]
38.4 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
dae = {'x': x, 'p': u, 'ode': ode}
intg_options = {'tf': 0.015}
intg = integrator('intg', 'rk', dae, intg_options)
# intg = integrator('intg', 'idas', dae, intg_options)


CasADi - 2023-05-01 17:11:11 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:499]


In [6]:
grf_positions_b = """[  0.1753,  0.08733,  -0.2813,  -0.1507,   0.1956,  -0.3318,   0.1618, -0.06079,  -0.3319,  -0.1972,   -0.145,  -0.2817]
[  0.1787,  0.09574,  -0.2847,   -0.154,   0.1911,  -0.3315,   0.1585, -0.06523,  -0.3316,  -0.1921,  -0.1373,  -0.2852]
[   0.183,   0.1049,  -0.2937,  -0.1564,   0.1873,  -0.3311,   0.1561,  -0.0691,  -0.3312,   -0.186,  -0.1289,   -0.294]
[  0.1865,   0.1131,  -0.3052,  -0.1588,    0.184,  -0.3305,   0.1537, -0.07238,  -0.3305,   -0.181,  -0.1213,  -0.3054]
[  0.1886,   0.1196,  -0.3167,  -0.1615,   0.1814,  -0.3299,    0.151, -0.07498,  -0.3299,  -0.1778,  -0.1153,  -0.3168]
[  0.1886,   0.1238,  -0.3258,  -0.1646,   0.1795,  -0.3296,   0.1479, -0.07683,  -0.3297,  -0.1769,  -0.1114,  -0.3258]
[  0.1859,    0.125,  -0.3299,  -0.1685,   0.1785,    -0.33,    0.144, -0.07781,  -0.3301,  -0.1793,  -0.1103,  -0.3299]
[  0.1811,    0.125,  -0.3314,  -0.1705,   0.1777,  -0.3277,   0.1425, -0.07927,  -0.3278,  -0.1841,  -0.1104,  -0.3314]
[  0.1762,   0.1251,  -0.3309,  -0.1678,   0.1754,   -0.318,   0.1468, -0.08318,  -0.3181,  -0.1891,  -0.1103,  -0.3309]
[  0.1711,   0.1252,  -0.3289,  -0.1613,    0.172,  -0.3039,   0.1557, -0.08912,  -0.3039,  -0.1942,  -0.1102,  -0.3289]
[  0.1656,   0.1253,  -0.3257,  -0.1522,   0.1677,  -0.2887,   0.1677, -0.09659,  -0.2887,  -0.1996,  -0.1101,  -0.3257]
[  0.1598,   0.1254,  -0.3218,  -0.1415,   0.1627,  -0.2755,   0.1817,  -0.1051,  -0.2755,  -0.2055,    -0.11,  -0.3218]
[  0.1534,   0.1253,  -0.3175,  -0.1304,   0.1574,  -0.2675,   0.1963,  -0.1142,  -0.2675,  -0.2119,    -0.11,  -0.3175]
[  0.1464,   0.1253,  -0.3131,  -0.1199,    0.152,  -0.2668,   0.2103,  -0.1234,  -0.2668,  -0.2188,  -0.1101,  -0.3131]
[   0.139,   0.1252,   -0.309,  -0.1108,   0.1468,   -0.272,   0.2227,  -0.1321,   -0.272,  -0.2262,  -0.1102,   -0.309]
[  0.1312,   0.1251,  -0.3054,  -0.1041,   0.1424,  -0.2804,   0.2324,  -0.1397,  -0.2804,  -0.2341,  -0.1102,  -0.3054]
[   0.123,   0.1252,  -0.3023,  -0.1007,   0.1389,  -0.2894,   0.2381,  -0.1456,  -0.2894,  -0.2422,  -0.1101,  -0.3023]
[  0.1145,   0.1255,  -0.2998,  -0.1015,   0.1369,  -0.2961,   0.2388,  -0.1493,  -0.2961,  -0.2508,  -0.1098,  -0.2998]
[  0.1056,   0.1261,  -0.2978,  -0.1076,   0.1366,  -0.2978,   0.2333,  -0.1502,  -0.2978,  -0.2596,  -0.1093,  -0.2978]
[  0.1009,   0.1264,  -0.2925,  -0.1168,   0.1374,  -0.2962,   0.2241,  -0.1494,  -0.2962,  -0.2646,  -0.1089,  -0.2925]
[   0.104,   0.1262,   -0.282,  -0.1259,   0.1384,   -0.295,    0.215,  -0.1484,   -0.295,  -0.2621,  -0.1091,   -0.282]
[  0.1136,   0.1254,   -0.269,   -0.135,   0.1395,   -0.294,   0.2059,  -0.1473,   -0.294,  -0.2534,  -0.1098,   -0.269]
[  0.1281,    0.124,  -0.2563,  -0.1438,   0.1405,  -0.2933,   0.1971,  -0.1463,  -0.2933,  -0.2401,   -0.111,  -0.2563]
[  0.1459,   0.1223,  -0.2466,  -0.1523,   0.1415,  -0.2929,   0.1886,  -0.1453,  -0.2929,  -0.2235,  -0.1126,  -0.2466]
[  0.1656,   0.1203,  -0.2426,  -0.1607,   0.1424,  -0.2926,   0.1802,  -0.1444,  -0.2926,  -0.2053,  -0.1145,  -0.2426]
[  0.1936,   0.1174,  -0.2463,  -0.1607,   0.1424,  -0.2926,   0.1802,  -0.1444,  -0.2926,  -0.1787,  -0.1172,  -0.2463]"""

grf_positions_b = np.array(grf_positions_b.replace("[", "").replace(
    "]", "").replace(",", "\n").split("\n"),
                           dtype=float)
grf_positions_b = grf_positions_b.reshape((-1, 12))
grf_positions_w = """[    0.4774,     0.1762,    0.05043,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,      0.105,   -0.05615,    0.04998]
[    0.4841,     0.1891,    0.04671,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1133,     -0.044,     0.0463]
[    0.4908,     0.2021,    0.03737,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1218,   -0.03171,    0.03704]
[    0.4968,     0.2135,    0.02522,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1292,   -0.02087,      0.025]
[    0.5015,     0.2227,    0.01308,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1352,   -0.01223,    0.01296]
[    0.5046,     0.2287,   0.003737,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1391,  -0.006508,   0.003704]
[    0.5058,     0.2309,          0,     0.1514,     0.2845, -4.708e-05,     0.4639,     0.0281, -0.0001344,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.1542,     0.2836,    0.00366,     0.4672,    0.02665,   0.003579,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.1619,     0.2813,    0.01293,     0.4765,    0.02265,    0.01286,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.1735,     0.2777,    0.02498,     0.4904,    0.01661,    0.02493,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,      0.188,     0.2733,    0.03702,     0.5079,   0.009042,      0.037,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2045,     0.2683,    0.04629,     0.5277,  0.0004514,    0.04629,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,      0.222,     0.2629,       0.05,     0.5487,   -0.00865,       0.05,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2395,     0.2576,     0.0463,     0.5697,   -0.01775,     0.0463,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,      0.256,     0.2526,    0.03704,     0.5895,   -0.02634,    0.03704,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2705,     0.2481,      0.025,      0.607,   -0.03391,      0.025,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2821,     0.2446,    0.01296,     0.6209,   -0.03995,    0.01296,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2898,     0.2423,   0.003704,     0.6301,   -0.04395,   0.003704,     0.1406,  -0.004438,          0]
[    0.5058,     0.2309,          0,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1406,  -0.004438,          0]
[    0.5102,     0.2305,   0.003704,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1448,  -0.004871,   0.003704]
[    0.5225,     0.2292,    0.01296,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1565,  -0.006066,    0.01296]
[    0.5411,     0.2273,      0.025,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1741,  -0.007872,      0.025]
[    0.5644,     0.2249,    0.03704,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.1963,   -0.01014,    0.03704]
[    0.5908,     0.2222,     0.0463,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.2214,   -0.01271,     0.0463]
[    0.6188,     0.2193,       0.05,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,      0.248,   -0.01543,       0.05]
[    0.6468,     0.2165,     0.0463,     0.2926,     0.2414,          0,     0.6335,    -0.0454,          0,     0.2746,   -0.01815,     0.0463]"""

grf_positions_w = np.array(grf_positions_w.replace("[", "").replace(
    "]", "").replace(",", "\n").split("\n"),
                           dtype=float)
grf_positions_w = grf_positions_w.reshape((-1, 12))
grf_plan = """[     -0,      -0,       0,  -44.99,  -44.99,     150,   -2.99,  -2.999,   10.01,      -0,      -0,       0]
[     -0,      -0,       0,  -18.13,  -34.12,   113.7,  -3.217,  -3.226,   10.76,      -0,      -0,       0]
[     -0,      -0,       0,   30.59,  -30.64,   102.1,  -9.669,  -9.686,   32.29,      -0,      -0,       0]
[     -0,      -0,       0,   33.33,  -33.35,   111.2,  -12.03,  -12.09,   40.31,      -0,      -0,       0]
[     -0,      -0,       0,    36.4,  -36.41,   121.4,   9.111,  -14.55,   48.52,      -0,      -0,       0]
[     -0,      -0,       0,   42.32,  -42.33,   141.1,   13.42,  -13.47,   44.91,      -0,      -0,       0]
[  3.001, -0.9456,   10.06,      -0,      -0,       0,      -0,      -0,       0,   6.534,  -6.502,   21.82]
[  5.736,    5.59,   19.17,      -0,      -0,       0,      -0,      -0,       0,   7.092,  -6.989,   23.68]
[  9.592,   7.586,   32.04,      -0,      -0,       0,      -0,      -0,       0,   10.85,  -10.46,   36.23]
[  12.38,   5.149,   41.35,      -0,      -0,       0,      -0,      -0,       0,    14.9,  -6.565,   49.75]
[  14.27,    2.92,   47.72,      -0,      -0,       0,      -0,      -0,       0,   18.84,  -1.544,   62.94]
[  16.11,   1.567,   54.05,      -0,      -0,       0,      -0,      -0,       0,    21.8,   1.348,   73.15]
[  18.32,  0.3076,   63.01,      -0,      -0,       0,      -0,      -0,       0,   19.85,   2.026,   79.71]
[  17.21,  -1.984,   75.18,      -0,      -0,       0,      -0,      -0,       0,   15.67,  0.5792,   81.42]
[  15.69,  -4.614,   85.17,      -0,      -0,       0,      -0,      -0,       0,   13.72,  -1.562,   79.38]
[  14.62,   -7.06,   91.96,      -0,      -0,       0,      -0,      -0,       0,   12.34,  -3.587,   75.25]
[  13.81,  -9.135,   95.82,      -0,      -0,       0,      -0,      -0,       0,   11.33,  -5.376,   69.98]
[  12.64,  -10.34,   97.05,      -0,      -0,       0,      -0,      -0,       0,   10.48,  -7.074,   64.63]
[     -0,      -0,       0,   1.644,  -10.16,   99.29,    1.77,  -9.783,   51.36,      -0,      -0,       0]
[     -0,      -0,       0,  -3.151,  -5.516,   90.38,  -2.531,  -4.767,   63.85,      -0,      -0,       0]
[     -0,      -0,       0,   -5.77,  -1.458,   81.79,  -5.499,  -1.146,   72.36,      -0,      -0,       0]
[     -0,      -0,       0,  -6.967,   1.512,   74.43,  -7.066,   1.401,   77.49,      -0,      -0,       0]
[     -0,      -0,       0,  -6.967,   3.241,   68.91,  -7.221,   2.968,   79.74,      -0,      -0,       0]
[     -0,      -0,       0,   -5.77,   3.695,   65.54,  -5.987,   3.471,   79.19,      -0,      -0,       0]
[     -0,      -0,       0,  -4.517,   2.613,   65.28,   -4.66,   2.462,   74.71,      -0,      -0,       0]"""
grf_plan = np.array(grf_plan.replace("[",
                                     "").replace("]",
                                                 "").replace(",",
                                                             "\n").split("\n"),
                    dtype=float)
grf_plan = grf_plan.reshape((-1, 12))
body_plan = """[    0.3021,    0.08888,     0.3317,     0.3966,    -0.3794,   -0.08081,     0.1943,     0.3215,   -0.04255,      -11.8,     -1.376,     0.4041]
[    0.3049,    0.09338,     0.3311,     0.2323,    -0.3993,   -0.08321,     0.1454,     0.2726,   -0.01687,     -5.882,     0.4581,   -0.00849]
[     0.307,    0.09747,     0.3309,     0.1435,    -0.3926,   -0.08162,     0.1221,     0.2318,    -0.0281,     -1.849,      1.715,  -0.005792]
[    0.3089,     0.1009,     0.3304,     0.1143,    -0.3671,   -0.07774,     0.1449,     0.1878,    -0.0285,     -1.201,      1.922,    -0.1764]
[     0.311,     0.1038,       0.33,    0.09599,    -0.3382,   -0.07703,     0.1682,     0.1382,   -0.01026,    -0.9989,       2.16,    -0.3548]
[    0.3136,     0.1058,     0.3299,    0.08178,    -0.3054,   -0.07935,     0.2179,    0.08257,    0.02808,     -0.677,      2.046,    -0.4304]
[    0.3168,     0.1071,     0.3303,    0.07286,    -0.2743,   -0.08347,     0.2787,    0.02165,    0.08401,      0.612,      1.998,    -0.5177]
[     0.321,     0.1074,     0.3315,     0.0836,    -0.2439,   -0.08925,     0.2892,    0.01352,   -0.02834,     -0.752,      1.937,    -0.4205]
[    0.3253,     0.1076,     0.3311,    0.07328,    -0.2144,   -0.09323,     0.3032,    0.01199,    -0.1287,     -0.656,      1.787,    -0.2762]
[    0.3299,     0.1078,     0.3292,    0.06392,    -0.1873,   -0.09545,     0.3255,   0.008852,    -0.2013,    -0.5252,      1.533,   -0.08485]
[    0.3348,     0.1079,     0.3262,      0.056,    -0.1643,   -0.09524,     0.3553,   0.007307,     -0.249,    -0.4224,      1.239,    0.04976]
[    0.3401,      0.108,     0.3224,    0.04937,    -0.1458,   -0.09344,     0.3914,   0.008809,    -0.2754,    -0.3388,     0.9645,     0.1204]
[     0.346,     0.1082,     0.3183,    0.04392,    -0.1314,   -0.09089,     0.4328,    0.01199,    -0.2836,    -0.2714,      0.715,     0.1507]
[    0.3525,     0.1083,     0.3141,    0.03949,    -0.1208,   -0.08814,     0.4744,    0.01454,     -0.275,    -0.2183,     0.5204,     0.1358]
[    0.3596,     0.1086,     0.3099,    0.03593,    -0.1131,   -0.08578,     0.5103,    0.01301,    -0.2512,    -0.1751,     0.4056,     0.1007]
[    0.3672,     0.1088,     0.3062,     0.0331,    -0.1071,   -0.08404,     0.5424,   0.006263,    -0.2187,    -0.1396,     0.3231,    0.07075]
[    0.3754,     0.1088,     0.3029,    0.03088,    -0.1023,   -0.08281,     0.5719,  -0.005361,    -0.1833,    -0.1103,     0.2587,    0.05035]
[     0.384,     0.1088,     0.3001,    0.02913,   -0.09841,   -0.08193,     0.5993,    -0.0212,    -0.1494,   -0.08618,     0.2089,    0.04294]
[    0.3929,     0.1084,     0.2979,    0.02777,    -0.0953,   -0.08119,     0.6246,   -0.04022,    -0.1201,   -0.06633,      0.191,    0.05759]
[    0.4023,     0.1078,     0.2961,    0.02668,   -0.09246,   -0.08024,     0.6283,   -0.06199,    -0.1027,    -0.0488,      0.142,    0.01254]
[    0.4117,     0.1069,     0.2945,    0.02593,   -0.09033,      -0.08,     0.6221,   -0.07322,   -0.08152,   -0.03604,     0.1074,  0.0004511]
[    0.4211,     0.1058,     0.2933,    0.02538,   -0.08872,   -0.07995,     0.6098,   -0.07606,   -0.06039,   -0.02566,    0.07889,  -0.001049]
[    0.4302,     0.1047,     0.2924,      0.025,   -0.08754,   -0.07993,     0.5945,   -0.07288,   -0.04169,   -0.01721,    0.05508, -0.0003251]
[    0.4391,     0.1036,     0.2918,    0.02474,   -0.08671,   -0.07992,      0.579,    -0.0661,   -0.02655,   -0.01032,    0.03524,  0.0002021]
[    0.4478,     0.1026,     0.2914,    0.02458,   -0.08619,    -0.0799,     0.5661,   -0.05828,    -0.0157,  -0.004712,    0.01997,  -5.49e-05]
[    0.4563,     0.1017,     0.2912,    0.02451,   -0.08589,    -0.0799,     0.5561,   -0.05274,   -0.01002, -3.713e-05,   0.005272, -6.666e-05]"""
body_plan = np.array(body_plan.replace("[", "").replace("]", "").replace(
    ",", "\n").split("\n"),
                     dtype=float)
body_plan = body_plan.reshape((-1, 12))

contact_plan = """0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
1 0 0 1 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0 
0 1 1 0"""
contact_plan = contact_plan.replace("\n", "").split(" ")
contact_plan = np.array(contact_plan, dtype=int).reshape((-1, 4))

ref_body_state = """[  0.3021,  0.08888,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3091,  0.08832,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3166,  0.08771,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3241,  0.08711,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3315,   0.0865,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[   0.339,   0.0859,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3465,  0.08529,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[   0.354,  0.08468,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3614,  0.08408,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3689,  0.08347,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3764,  0.08287,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3839,  0.08226,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3913,  0.08166,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.3988,  0.08105,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4063,  0.08045,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4138,  0.07984,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4212,  0.07924,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4287,  0.07863,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4362,  0.07803,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4437,  0.07742,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4511,  0.07681,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4586,  0.07621,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4661,   0.0756,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4736,    0.075,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[   0.481,  0.07439,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]
[  0.4885,  0.07379,     0.27,        0,        0, -0.08081,   0.4983, -0.04036,        0,        0,        0,        0]"""
ref_body_state = np.array(ref_body_state.replace("[",
                                                 "").replace("]", "").replace(
                                                     ",", "\n").split("\n"),
                          dtype=float)
ref_body_state = ref_body_state.reshape((-1, 12))

next_state = """[  0.3057]
[ 0.09647]
[  0.3305]
[  0.1079]
[ -0.4087]
[-0.08604]
[  0.1049]
[  0.3057]
[-0.05265]
[  -13.78]
[ -0.8608]
[ -0.1516]"""
next_state = np.array(next_state.replace("[", "").replace("]", "").replace(
    ",", "\n").split("\n"),
                      dtype=float)
next_state = next_state.reshape((-1, 12))


In [7]:
x_data = np.zeros((2, 12))
x_next = np.copy(body_plan[0, :])
# x_next[9:12] = [0, 0, 0]
for i in range(1):
    x_data[i, :] = x_next
    foot_pose = grf_positions_w[i, :]
    grf = grf_plan[i, :]
    x_next = intg(x0=x_next, p=np.concatenate([foot_pose, grf]))["xf"].full().T
x_data[i + 1, :] = x_next

print("body plan")
with np.printoptions(precision=3, suppress=True):
    display(pandas.DataFrame(body_plan[0:2, :]))
print("real")
with np.printoptions(precision=3, suppress=True):
    display(pandas.DataFrame(np.vstack([body_plan[0, :].T, next_state])))
print("comparison")
with np.printoptions(precision=3, suppress=True):
    display(pandas.DataFrame(body_plan[0, :]).T)
    display(pandas.DataFrame(np.vstack([x_next, next_state])))


body plan


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.3021,0.08888,0.3317,0.3966,-0.3794,-0.08081,0.1943,0.3215,-0.04255,-11.800,-1.3760,0.40410
1,0.3049,0.09338,0.3311,0.2323,-0.3993,-0.08321,0.1454,0.2726,-0.01687,-5.882,0.4581,-0.00849


real


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.3021,0.08888,0.3317,0.3966,-0.3794,-0.08081,0.1943,0.3215,-0.04255,-11.80,-1.3760,0.4041
1,0.3057,0.09647,0.3305,0.1079,-0.4087,-0.08604,0.1049,0.3057,-0.05265,-13.78,-0.8608,-0.1516


comparison


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.3021,0.08888,0.3317,0.3966,-0.3794,-0.08081,0.1943,0.3215,-0.04255,-11.8,-1.376,0.4041


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.304622,0.09331,0.331268,0.26813,-0.386145,-0.078823,0.14192,0.26911,-0.015017,-5.272753,0.618209,0.237574
1,0.305700,0.09647,0.330500,0.10790,-0.408700,-0.086040,0.10490,0.30570,-0.052650,-13.780000,-0.860800,-0.151600


In [8]:
grf_plan.shape
grf_positions_w.shape


(26, 12)

In [9]:
print(
    substitute([inv(Iner) @ skew(omega) @ I @ omega], [omega, theta],
               [[4.813, 1.928, 0.9040], [-0.5392, 0.1516, -0.1343]]))
print(
    substitute([inv(Iner) @ skew(omega) @ Iner @ omega], [omega, theta],
               [[4.813, 1.928, 0.9040], [-0.5392, 0.1516, -0.1343]]))


NameError: name 'Iner' is not defined

In [ ]:
ref_body_state.shape


(26, 12)

In [ ]:
pose = body_plan[0, 0:3]
for i in range(4):
    print(-pose + grf_positions_w[0, i * 3:(i + 1) * 3])


[ 0.1753   0.08732 -0.28127]
[-0.1507      0.19562    -0.33174708]
[ 0.1618    -0.06078   -0.3318344]
[-0.1971  -0.14503 -0.28172]


In [ ]:
x_data[24, :]


IndexError: index 24 is out of bounds for axis 0 with size 2

In [11]:
x_dim = 12
u_dim = 12
foot_count = 4
N = 8
dt = 0.03
mu = 0.3
opti = casadi.Opti()
x = opti.variable(x_dim * N)
u = opti.variable(u_dim * (N - 1))
x0 = opti.parameter(x_dim)
x_ref = opti.parameter(x_dim * N)
x_foots = opti.parameter(N, u_dim * N)
contacts = opti.parameter(N, foot_count)

u_ref = np.zeros(u_dim * (N - 1))
for i in range(N - 1):
    for leg in range(foot_count):
        u_ref[i * u_dim + leg * 3 + 2] = 13.74 * 9.81 / 2


In [12]:
u_ref


array([ 0.    ,  0.    , 67.3947,  0.    ,  0.    , 67.3947,  0.    ,
        0.    , 67.3947,  0.    ,  0.    , 67.3947,  0.    ,  0.    ,
       67.3947,  0.    ,  0.    , 67.3947,  0.    ,  0.    , 67.3947,
        0.    ,  0.    , 67.3947,  0.    ,  0.    , 67.3947,  0.    ,
        0.    , 67.3947,  0.    ,  0.    , 67.3947,  0.    ,  0.    ,
       67.3947,  0.    ,  0.    , 67.3947,  0.    ,  0.    , 67.3947,
        0.    ,  0.    , 67.3947,  0.    ,  0.    , 67.3947,  0.    ,
        0.    , 67.3947,  0.    ,  0.    , 67.3947,  0.    ,  0.    ,
       67.3947,  0.    ,  0.    , 67.3947,  0.    ,  0.    , 67.3947,
        0.    ,  0.    , 67.3947,  0.    ,  0.    , 67.3947,  0.    ,
        0.    , 67.3947,  0.    ,  0.    , 67.3947,  0.    ,  0.    ,
       67.3947,  0.    ,  0.    , 67.3947,  0.    ,  0.    , 67.3947])

In [13]:
ca_contacts = []
for i in range(N):
    a = []
    z = MX.zeros((3, 3))
    for j in range(4):
        a.append(np.eye(3) * (1 - contacts[i, j]))

    np_contacts = hcat([
        hcat([a[0], z, z, z]),
        hcat([z, a[1], z, z]),
        hcat([z, z, a[2], z]),
        hcat([z, z, z, a[3]])
    ])
    ca_contacts.append(np_contacts)
ca_contacts = vcat(ca_contacts)

# ca_contacts @ u
hcat([a[0], z, z, z])


MX(@1=zeros(3x3), horzcat((
[[1, 0, 0], 
 [0, 1, 0], 
 [0, 0, 1]]*(1-opti0_p_4[7])), @1, @1, @1))

In [14]:
dx = x - x_ref
du = u - u_ref
# weights = [1, 1, 1,
#            0.1, 0.1, 0.1,
#            0.1, 0.1, 0.1,
#            0.01, 0.01, 0.01]

# weights = [1.1665, 1.1665, 1.1665,
#             0.11665,  0.11665,  0.11665,
#            0.02333, 0.02333, 0.04666,
#            0.011665, 0.011665, 0.002333]

weights = [1., 1., 10., 0.1, 0.1, 0.1, 0.02, 0.02, 0.04, 0.01, 0.01, 0.002]

Q = np.diag(weights * N)
R = np.diag([2.5e-05] * 12 * (N - 1))

weighted_sum = dx.T @ Q @ dx + du.T @ R @ du
opti.minimize(weighted_sum)
# opti.subject_to(ca_contacts @ u == 0)
for i in range(N - 1):
    for j in range(foot_count):

        fz = u[i * 12 + j * 3 + 2]
        opti.subject_to(u[i * 12 + j * 3 + 1] <= mu * fz)
        opti.subject_to(u[i * 12 + j * 3 + 1] >= -mu * fz)
        opti.subject_to(u[i * 12 + j * 3] <= mu * fz)
        opti.subject_to(u[i * 12 + j * 3] >= -mu * fz)

        for k in range(3):
            opti.subject_to((1 - contacts[i, j]) * u[i * 12 + j * 3 + k] == 0)

opti.subject_to(x[0:x_dim] == x0)
for k in range(1, N):
    opti.subject_to(x[k * x_dim:(k + 1) * x_dim] == intg(
        x0=x[(k - 1) * x_dim:k * x_dim],
        p=vcat(
            [x_foots[(k - 1) * u_dim:k * u_dim], u[(k - 1) * u_dim:k *
                                                   u_dim]]))['xf'])

opti.solver('ipopt')
mpc = opti.to_function("MPC", [x0, x_ref, x_foots, contacts], [u, x],
                       ["x0", "x_ref", "x_foots", "contacts"], ["u_opt", "x"])


In [15]:
x

MX(opti0_x_1)

In [ ]:
x0_real = body_plan[0, :]
x_ref_real = ref_body_state[0:N, :].reshape((-1))
x_foots_real = grf_positions_w[0:N, :]
contacts_real = contact_plan[0:N, :]
mpc_res = mpc(x0_real, x_ref_real, x_foots_real, contacts_real)


This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:     1419
Number of nonzeros in inequality constraint Jacobian.:      224
Number of nonzeros in Lagrangian Hessian.............:     9234

Total number of variables............................:      180
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      180
Total number of inequality constraints...............:      112
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      112

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0142216e+01 9.49e-01 5.40e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [ ]:
u_res = mpc_res[0].full().reshape((-1, 12))
for i in u_res:
    print(i.reshape((-1, 3)))


[[-5.85731631e-18  2.31150822e-18 -8.82299993e-09]
 [-1.57010616e+01  1.57010633e+01  5.23368826e+01]
 [ 1.37687323e+01 -1.37687334e+01  4.58957866e+01]
 [ 6.12188758e-18 -3.94287237e-18 -8.82299994e-09]]
[[ 6.47970976e-18 -4.41852958e-18 -8.82299994e-09]
 [-1.14807419e+00  1.14807541e+00  3.82692091e+00]
 [-1.51211955e+01  1.51211975e+01  5.04039951e+01]
 [-7.15656599e-18  2.45235371e-18 -8.82299993e-09]]
[[-3.02413690e-18 -2.12675126e-18 -8.82299993e-09]
 [-1.21106486e+01  1.21106531e+01  4.03688464e+01]
 [-4.40258231e+00  9.65038505e+00  3.21679609e+01]
 [-7.83663990e-19 -1.50464676e-18 -8.82299993e-09]]
[[ 6.86390644e-19 -2.20899670e-18 -8.82299993e-09]
 [-8.01151474e+00  8.01151514e+00  2.67050532e+01]
 [-1.65798481e+01  1.65798486e+01  5.52661654e+01]
 [ 1.24218309e-18 -3.40271209e-18 -8.82299993e-09]]
[[-3.98792166e+00  4.61172202e+00  2.55049467e+01]
 [ 3.01906057e-19 -6.90268404e-19 -8.82299992e-09]
 [ 1.07277083e-18 -1.36563910e-18 -8.82299992e-09]
 [-5.15307178e+00  5.795733

In [ ]:
x_res = mpc_res[1].full().reshape((-1, 12))
for i, item in enumerate(x_res):
    print(item.reshape((-1, 3)))


[[ 0.03171   0.1658    0.2613  ]
 [-0.002086 -0.0262    0.9489  ]
 [ 0.2355    0.341     0.01228 ]
 [-0.08213   0.2287   -0.1531  ]]
[[ 0.03405797  0.16921703  0.26128977]
 [-0.0045755  -0.02369225  0.94725499]
 [ 0.23409365  0.34240635 -0.01432606]
 [-0.61826512 -0.33954052 -0.17569042]]
[[ 0.0363397   0.1727003   0.26085336]
 [-0.01173475 -0.01982483  0.94516368]
 [ 0.22225284  0.35424716 -0.07295669]
 [-0.84826447 -0.36986602 -0.24301586]]
[[ 0.03850214  0.17632196  0.25989725]
 [-0.00950359 -0.01059672  0.94128368]
 [ 0.21023448  0.37008489 -0.11826426]
 [-0.38484437  1.81216043 -0.53636856]]
[[ 4.05149938e-02  1.80112297e-01  2.58522403e-01]
 [ 1.60134556e-03  4.86384110e-03  9.35285214e-01]
 [ 1.92336832e-01  3.87982536e-01 -1.56705436e-01]
 [-8.01123347e-01  1.80491320e+00 -6.63462073e-01]]
[[ 0.0424051   0.18402999  0.25665041]
 [ 0.00194167  0.02055493  0.92915243]
 [ 0.185684    0.3955571  -0.21769406]
 [-1.68372017  0.12097701 -0.55895825]]
[[ 0.0442177   0.18802858  0.25433

In [ ]:
x_res = mpc_res[1].full().reshape((-1, 12))
for i, item in enumerate(x_res):
    print(
        item.reshape((-1, 3)) -
        x_ref_real.reshape((-1, 12))[i].reshape((-1, 3)))


[[-1.36543554e-13  1.18571819e-13 -8.70000000e-03]
 [-2.08600000e-03 -2.62000000e-02 -3.34177130e-14]
 [-5.58000000e-02 -6.54000000e-02  1.22800000e-02]
 [-8.21300000e-02  2.28700000e-01 -1.53100000e-01]]
[[-0.00289203 -0.00388297 -0.00871023]
 [-0.0045755  -0.02369225 -0.00164501]
 [-0.05720635 -0.06399365 -0.01432606]
 [-0.61826512 -0.33954052 -0.17569042]]
[[-0.0093503  -0.0125997  -0.00914664]
 [-0.01173475 -0.01982483 -0.00373632]
 [-0.06904716 -0.05215284 -0.07295669]
 [-0.84826447 -0.36986602 -0.24301586]]
[[-0.01592786 -0.02117804 -0.01010275]
 [-0.00950359 -0.01059672 -0.00761632]
 [-0.08106552 -0.03631511 -0.11826426]
 [-0.38484437  1.81216043 -0.53636856]]
[[-2.26550062e-02 -2.95877032e-02 -1.14775973e-02]
 [ 1.60134556e-03  4.86384110e-03 -1.36147857e-02]
 [-9.89631679e-02 -1.84174638e-02 -1.56705436e-01]
 [-8.01123347e-01  1.80491320e+00 -6.63462073e-01]]
[[-0.0294949  -0.03787001 -0.01334959]
 [ 0.00194167  0.02055493 -0.01974757]
 [-0.105616   -0.0108429  -0.21769406]
 [

In [ ]:
np.array([2.49983333e-05] * 12 * (N - 1)).shape


(84,)

In [ ]:
ref_body_state[0:N, :].reshape((-1))


array([3.171e-02, 1.658e-01, 2.700e-01, 0.000e+00, 0.000e+00, 9.489e-01,
       2.913e-01, 4.064e-01, 0.000e+00, 0.000e+00, 0.000e+00, 1.492e-22,
       3.695e-02, 1.731e-01, 2.700e-01, 0.000e+00, 0.000e+00, 9.489e-01,
       2.913e-01, 4.064e-01, 0.000e+00, 0.000e+00, 0.000e+00, 1.492e-22,
       4.569e-02, 1.853e-01, 2.700e-01, 0.000e+00, 0.000e+00, 9.489e-01,
       2.913e-01, 4.064e-01, 0.000e+00, 0.000e+00, 0.000e+00, 1.492e-22,
       5.443e-02, 1.975e-01, 2.700e-01, 0.000e+00, 0.000e+00, 9.489e-01,
       2.913e-01, 4.064e-01, 0.000e+00, 0.000e+00, 0.000e+00, 1.492e-22,
       6.317e-02, 2.097e-01, 2.700e-01, 0.000e+00, 0.000e+00, 9.489e-01,
       2.913e-01, 4.064e-01, 0.000e+00, 0.000e+00, 0.000e+00, 1.492e-22,
       7.190e-02, 2.219e-01, 2.700e-01, 0.000e+00, 0.000e+00, 9.489e-01,
       2.913e-01, 4.064e-01, 0.000e+00, 0.000e+00, 0.000e+00, 1.492e-22,
       8.064e-02, 2.340e-01, 2.700e-01, 0.000e+00, 0.000e+00, 9.489e-01,
       2.913e-01, 4.064e-01, 0.000e+00, 0.000e+00, 

In [ ]:
grf_plan[0, :] - u_res[0]


array([ 5.85731631e-18, -2.31150822e-18,  8.82299993e-09,  2.10490616e+01,
       -1.47604633e+01,  1.98431174e+01, -1.00097323e+01,  1.96057334e+01,
        2.23642134e+01, -6.12188758e-18,  3.94287237e-18,  8.82299994e-09])

In [ ]:
grf_plan[0, :]


array([-0.    , -0.    ,  0.    ,  5.348 ,  0.9406, 72.18  ,  3.759 ,
        5.837 , 68.26  , -0.    , -0.    ,  0.    ])

In [ ]:
QQ = """[[1.1665   0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.      ]
 [0.       1.1665   0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.      ]
 [0.       0.       1.1665   0.       0.       0.       0.       0.
  0.       0.       0.       0.      ]
 [0.       0.       0.       0.11665  0.       0.       0.       0.
  0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.11665  0.       0.       0.
  0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.11665  0.       0.
  0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.02333  0.
  0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.02333
  0.       0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.04666  0.       0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.011665 0.       0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.011665 0.      ]
 [0.       0.       0.       0.       0.       0.       0.       0.
  0.       0.       0.       0.002333]]"""

# QQ = QQ.replace("[", "").replace(
#     "]", "").replace("       ", ",").replace("   ", ",").replace(" ", ",").replace("\n", ",")
# print(QQ)
# QQ = np.array(QQ.replace("[", "").replace(
#     "]", "").replace("   ", ",").replace(" ", ",").replace("       ", ",").replace("\n", ",").split(","), dtype=float)
# QQ = QQ.reshape((-1, 12))
# QQ
"""[[2.49983333e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.49983333e-05 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.49983333e-05 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.49983333e-05
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.49983333e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.49983333e-05 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 2.49983333e-05 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.49983333e-05
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.49983333e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 2.49983333e-05 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 2.49983333e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.49983333e-05]]"""


'[[2.49983333e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00\n  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00\n  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]\n [0.00000000e+00 2.49983333e-05 0.00000000e+00 0.00000000e+00\n  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00\n  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]\n [0.00000000e+00 0.00000000e+00 2.49983333e-05 0.00000000e+00\n  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00\n  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]\n [0.00000000e+00 0.00000000e+00 0.00000000e+00 2.49983333e-05\n  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00\n  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]\n [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00\n  2.49983333e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00\n  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]\n [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.

In [ ]:
QQ = np.array(QQ.replace("[", "").replace("]", "").replace("\n", " "))
QQ


array('1.1665   0.       0.       0.       0.       0.       0.       0.   0.       0.       0.       0.        0.       1.1665   0.       0.       0.       0.       0.       0.   0.       0.       0.       0.        0.       0.       1.1665   0.       0.       0.       0.       0.   0.       0.       0.       0.        0.       0.       0.       0.11665  0.       0.       0.       0.   0.       0.       0.       0.        0.       0.       0.       0.       0.11665  0.       0.       0.   0.       0.       0.       0.        0.       0.       0.       0.       0.       0.11665  0.       0.   0.       0.       0.       0.        0.       0.       0.       0.       0.       0.       0.02333  0.   0.       0.       0.       0.        0.       0.       0.       0.       0.       0.       0.       0.02333   0.       0.       0.       0.        0.       0.       0.       0.       0.       0.       0.       0.   0.04666  0.       0.       0.        0.       0.       0.       0.       0.     